In [ ]:
import xarray as xr
import fsspec

In [ ]:
%%time
fs = fsspec.filesystem('s3', anon=True)
flist = fs.glob("s3://noaa-goes16/ABI-L2-SSTF/2020/210/*/*.nc")

In [ ]:
flist

In [ ]:
fs.size(flist[0])/1e6

In [ ]:
len(flist)

In [ ]:
client

In [ ]:
%%time
file_objs = [fs.open(file) for file in flist]
ds = xr.open_mfdataset(file_objs[:5], parallel=True, combine='nested', concat_dim='t', 
                       data_vars='minimal', coords='minimal', compat='override')

In [ ]:
def preprocessing(ds): 
    return ds.expand_dims(dim='t')

In [ ]:
ds = xr.open_mfdataset([file_objs[0]], preprocess=preprocessing)

In [ ]:
ds.SST

In [ ]:
print(ds.SST)

#### Try Local Cluster

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
#client.close()

#### Try Remote Cluster

In [ ]:
import sys, os
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

worker_max=4
client, cluster=ebd.start_dask_cluster(profile='esip-qhub',worker_max=worker_max,
                                      worker_profile='Pangeo Worker',
                                      propagate_env=False, adaptive_scaling=False)  

In [ ]:
client

In [ ]:
%%time
ds = xr.open_mfdataset(file_objs, parallel=True, concat_dim='t', combine='nested',
                       coords='minimal', data_vars='minimal', compat='override', engine='h5netcdf')

In [ ]:
%%time
ds = xr.open_mfdataset(file_objs, parallel=True, preprocess=preprocessing, concat_dim='t', combine='by_coords',
                       coords='minimal', data_vars='minimal', compat='override', engine='h5netcdf')

#### No Client

In [ ]:
%%time 
ds = xr.open_mfdataset(file_objs, combine='by_coords')

In [ ]:
%%time
ds = xr.open_mfdataset(file_objs, parallel=True, engine='h5netcdf',
                       combine='by_coords', coords='minimal', 
                       data_vars='minimal', compat='override')